<a href="https://colab.research.google.com/github/Disha-Sikka/Maize_Disease_Classification/blob/main/Maize_Disease_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!kaggle competitions download -c ssd-3-l-maiize-disease-classification

 57% 92.0M/161M [00:00<00:00, 964MB/s]
100% 161M/161M [00:00<00:00, 976MB/s] 


In [6]:
import zipfile
zip_ref= zipfile.ZipFile('/content/ssd-3-l-maiize-disease-classification.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [7]:
import os

BASE_DIR = "/content/maize_dataset_split"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")
CHECKPOINT_DIR = os.path.join(BASE_DIR, "checkpoints")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

NUM_CLASSES = 4
BATCH_SIZE = 32
EPOCHS = 15
LR = 0.001
LABELS = ['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

In [8]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm


In [9]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

val_test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [10]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(), nn.MaxPool2d(2),  # 64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(), nn.MaxPool2d(2),  # 32x32

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(), nn.MaxPool2d(2),  # 16x16

            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, NUM_CLASSES)
        )

    def forward(self, x):
        return self.model(x)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


In [11]:
train_losses, val_losses, val_accuracies = [], [], []

for epoch in range(EPOCHS):
    model.train()
    epoch_train_loss = 0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()

    train_losses.append(epoch_train_loss / len(train_loader))

    # ✅ Validation
    model.eval()
    val_loss, correct = 0, 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            val_loss += criterion(outputs, labels).item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(correct / total)
    print(f"✅ Epoch {epoch+1}: Val Acc = {100*correct/total:.2f}%, Train Loss = {train_losses[-1]:.4f}")

    # Save checkpoint
    torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f"model_epoch{epoch+1}.pth"))


Epoch 1/15: 100%|██████████| 92/92 [00:13<00:00,  7.02it/s]


✅ Epoch 1: Val Acc = 82.48%, Train Loss = 0.7614


Epoch 2/15: 100%|██████████| 92/92 [00:11<00:00,  7.76it/s]


✅ Epoch 2: Val Acc = 82.64%, Train Loss = 0.4747


Epoch 3/15: 100%|██████████| 92/92 [00:12<00:00,  7.50it/s]


...

✅ Epoch 13: Val Acc = 91.40%, Train Loss = 0.1907


Epoch 14/15: 100%|██████████| 92/92 [00:11<00:00,  7.72it/s]


✅ Epoch 14: Val Acc = 90.13%, Train Loss = 0.1763


Epoch 15/15: 100%|██████████| 92/92 [00:11<00:00,  7.69it/s]


✅ Epoch 15: Val Acc = 90.45%, Train Loss = 0.1590
